## Custom SageMaker Studio Kernel Demo
First, open SageMaker Studio (**this notebook should run inside Studio**).

### 1. Get your the IAM Role associated with this user inside Studio:

In [ ]:
import os
import sagemaker
import boto3

In [ ]:
sm_client = boto3.client("sagemaker")

In [ ]:
sess = sagemaker.session.Session()
sess.get_caller_identity_arn()

### 2. Go to the IAM console and choose `Roles`. Find the role above.

Add a new policy to the role, click on `Attach Policies`, then `Create Policy`.

Choose the JSON editor, and substitute everything by:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "ecr:CreateRepository",
                "ecr:BatchGetImage",
                "ecr:CompleteLayerUpload",
                "ecr:DescribeImages",
                "ecr:DescribeRepositories",
                "ecr:UploadLayerPart",
                "ecr:ListImages",
                "ecr:InitiateLayerUpload",
                "ecr:BatchCheckLayerAvailability",
                "ecr:GetDownloadUrlForLayer",
                "ecr:PutImage"
            ],
            "Resource": "arn:aws:ecr:*:*:repository/smstudio*"
        },
        {
            "Effect": "Allow",
            "Action": "ecr:GetAuthorizationToken",
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "codebuild:DeleteProject",
                "codebuild:CreateProject",
                "codebuild:BatchGetBuilds",
                "codebuild:StartBuild"
            ],
            "Resource": "arn:aws:codebuild:*:*:project/sagemaker-studio*"
        },
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::*:role/*",
            "Condition": {
                "StringLikeIfExists": {
                    "iam:PassedToService": "codebuild.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": "sagemaker:UpdateDomain",
            "Resource": "*"
        }
    ]
}
```

Name the IAM policy `SageMakerStudioBuildCustomKernel`.

### 3. Go back to the IAM role and attach this new created policy.

Still inside the IAM Role, go to the tab `Trust relationships`, click on `Edit trust relationship`.

Add the following:
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      ...
    },
    
    ...
    
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "codebuild.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```

### 4. Create Dockerfile

In [ ]:
%%writefile Dockerfile
FROM tensorflow/tensorflow:2.3.0
RUN apt-get update 
RUN apt-get install -y git
RUN pip install --upgrade pip
RUN pip install ipykernel && \
    python -m ipykernel install --sys-prefix && \
    pip install --quiet --no-cache-dir \
    'boto3>1.0<2.0' \
    'sagemaker>2.0<3.0'
    
WORKDIR /root
COPY train.py train.py

### 5. Create training script

In [ ]:
%%writefile train.py
import tensorflow as tf
import os 

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test, y_test)

## 6. Create some environment variables to help

In [ ]:
def set_env_var(key, value):
    os.environ[key]=value
    print(f"Environment variable:  {key}={value}")

In [ ]:
set_env_var("APP_CONF_NAME", "custom-tf2")

In [ ]:
set_env_var("KERNEL_IMAGE_NAME", "tf2kernel")

In [ ]:
set_env_var("AWS_ACCOUNT_ID", sess.account_id())

In [ ]:
set_env_var("AWS_REGION", sess.boto_region_name)

In [ ]:
domain_id = sm_client.list_domains()['Domains'][0]['DomainId']
set_env_var("STUDIO_DOMAIN_ID", domain_id)

In [ ]:
set_env_var("ROLE_ARN", sess.get_caller_identity_arn())

### 7. Create image configuration of file system for Studio domain

In [ ]:
import json, os

In [ ]:
with open('app-image-config-input.json', 'w') as f:
    data = {
                "AppImageConfigName": os.environ["APP_CONF_NAME"],
                "KernelGatewayImageConfig": {
                    "KernelSpecs": [
                        {
                            "Name": "python3",
                            "DisplayName": "Python 3"
                        }
                    ],
                    "FileSystemConfig": {
                        "MountPath": "/root/data",
                        "DefaultUid": 0,
                        "DefaultGid": 0
                    }
                }
            }
    json.dump(data, f)

### 8. Create App configuration for Studio

In [ ]:
with open('default-user-settings.json', 'w') as f:
    data =  {
              "DefaultUserSettings": {
                "KernelGatewayAppSettings": {
                  "CustomImages": [
                      {
                               "ImageName": os.environ["KERNEL_IMAGE_NAME"],
                               "AppImageConfigName": os.environ["APP_CONF_NAME"]
                            }
                        ]
                    }
                }
            }
    json.dump(data, f)

### 9. Install the tool Sagemaker Studio Image Build (for building Docker images within Studio)

In [ ]:
!pip install sagemaker-studio-image-build

### 10. Build Docker image

In [ ]:
!sm-docker build . --repository smstudio-custom:$KERNEL_IMAGE_NAME

### 11. Go to the ECR console and check if the ECR repository called `smstudio-custom` is there and the image tag `tf2kernel` is also there

Click here:
[https://console.aws.amazon.com/ecr/](https://us-west-1.console.aws.amazon.com/ecr/repositories/private/795875386142/smstudio-custom?region=us-west-1)

### 12. Publish image to Studio

In [ ]:
!aws --region ${AWS_REGION} sagemaker create-image --image-name ${KERNEL_IMAGE_NAME} --role-arn ${ROLE_ARN}

In [ ]:
!aws --region ${AWS_REGION} sagemaker create-image-version --image-name ${KERNEL_IMAGE_NAME} --base-image "${AWS_ACCOUNT_ID}.dkr.ecr.${AWS_REGION}.amazonaws.com/smstudio-custom:${KERNEL_IMAGE_NAME}"  

In [ ]:
## Create AppImageConfig for this image
!aws --region ${AWS_REGION} sagemaker create-app-image-config --cli-input-json file://app-image-config-input.json

In [ ]:
## Update the Domain, providing the Image and AppImageConfig
!aws --region ${AWS_REGION} sagemaker update-domain --domain-id ${STUDIO_DOMAIN_ID} --cli-input-json file://default-user-settings.json

### 13. Test it!

 Click on the top in the `File` > `New Notebook`.
- Select the new Kernel called `tf2kernel`.
- Run:
```
import tensorflow as tf
tf.__version__
```

It should output:
`2.3.0`

Should be like this:

![custom-kernel.png](custom-kernel.png)

Run: `!cat /root/train.py`
It should output: our training script that we saved in the docker image.

Should be like this:

![script-saved-in-container.png](script-saved-in-container.png)